In [1]:
from dotenv import get_key
from pymongo import MongoClient
import pymongo
import psycopg2 # Remover
from pprint import pprint  # Imprimir bsons formatados
from utils import *
import pandas as pd

STRING = get_key('.env', 'MG_STRING')
NAME = get_key('.env', 'PG_NAME')
USER = get_key('.env', 'PG_USER')
PASSWORD = get_key('.env', 'PG_PASSWORD')
HOST = get_key('.env', 'PG_HOST')
PORT = get_key('.env', 'PG_PORT')

## Funções para acesso ao MongoDB

In [2]:
def create_mongo_connection():
	"""Cria uma conexão com o banco de dados MongoDB. Retorna o cliente.
	AVISO: Se a conexão não funcionar, verifique no site do atlas se seu IP atual está autorizado a se conectar."""
	try:
		client = MongoClient(STRING, server_api=pymongo.server_api.ServerApi(version="1"))
		client.admin.command("ping")
		print("---- Conexão bem sucedida ----")
	except Exception as e:
		raise Exception(
			"The following error occurred: ", e)
	return client

def end_mongo_connection(client):
	client.close()
	print("---- Conexão encerrada -----")

## Criação de tabelas no banco

In [3]:
def create_pais_collection(db, v_action, v_level):
    schema = {
        "$jsonSchema": {
            "bsonType": "object",
            "required": ["sigla", "nome", "continente", "grupos", "estatisticas"],
            "properties": {
                "sigla": {"bsonType": "string"},
                "nome": {"bsonType": "string"},
                "continente": {"bsonType": "string"},
                "grupos": {
                    "bsonType": "array",
                    "items": {"bsonType": "string"}
                },
                "estatisticas": {
                    "bsonType": "array",
                    "items": {
                        "bsonType": "object",
                        "required": ["ano", "pib", "geografia"],
                        "properties": {
                            "ano": {"bsonType": "int"},
                            "pib": {"bsonType": "double"},
                            "geografia": {
                                "bsonType": "array",
                                "items": {
                                    "bsonType": "object",
                                    "required": ["area_tipo", "num_habitantes", "area_km2"],
                                    "properties": {
                                        "area_tipo": {"bsonType": "string"},
                                        "num_habitantes": {"bsonType": "int"},
                                        "area_km2": {"bsonType": "double"}
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
    }

    try:
        db.create_collection("pais", validator=schema, validationAction=v_action, validationLevel=v_level)
    except Exception as e:
        print(f"Erro ao criar a coleção: {e}")


In [4]:
def create_emissao_gas_collection(db, v_action, v_level):
    schema = {
        "$jsonSchema": {
            "bsonType": "object",
            "required": ["nome", "formula", "medicoes"],
            "properties": {
                "nome": {"bsonType": "string"},
                "formula": {"bsonType": "string"},
                "medicoes": {
                    "bsonType": "array",
                    "items": {
                        "bsonType": "object",
                        "required": ["cod_pais", "ano", "valor_mtco2"],
                        "properties": {
                            "cod_pais": {"bsonType": "string"},
                            "ano": {"bsonType": "int"},
                            "valor_mtco2": {"bsonType": "double"}
                        }
                    }
                }
            }
        }
    }

    try:
        db.create_collection("emissaoGas", validator=schema, validationAction=v_action, validationLevel=v_level)
    except Exception as e:
        print(f"Erro ao criar a coleção: {e}")


In [5]:
def create_energia_collection(db, v_action, v_level):
    schema = {
        "$jsonSchema": {
            "bsonType": "object",
            "required": ["tipo_interacao", "unidade", "fontes"],
            "properties": {
                "tipo_interacao": {"bsonType": "string"},
                "unidade": {"bsonType": "string"},
                "fontes": {
                    "bsonType": "array",
                    "items": {
                        "bsonType": "object",
                        "required": ["nome", "medicoes"],
                        "properties": {
                            "nome": {"bsonType": "string"},
                            "medicoes": {
                                "bsonType": "array",
                                "items": {
                                    "bsonType": "object",
                                    "required": ["cod_pais", "ano", "valor"],
                                    "properties": {
                                        "cod_pais": {"bsonType": "string"},
                                        "ano": {"bsonType": "int"},
                                        "valor": {"bsonType": "double"}
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
    }

    try:
        db.create_collection("energia", validator=schema, validationAction=v_action, validationLevel=v_level)
    except Exception as e:
        print(f"Erro ao criar a coleção: {e}")


In [6]:
def create_tables(db, col, v_action="warn", v_level="moderate"):
    if v_action not in ["warn", "error"]:
        raise ValueError("Ação de validação inválida. Use 'warn' ou 'error'.")
    if v_level not in ["strict", "moderate"]:
        raise ValueError("Nível de validação inválido. Use 'off', 'strict' ou 'moderate'.")
    if col == 'pais':
        create_pais_collection(db, v_action, v_level)
    elif col == 'emissaoGas':
        create_emissao_gas_collection(db, v_action, v_level)
    elif col == 'energia':
        create_energia_collection(db, v_action, v_level)
    else:
        raise ValueError(f"Nome de coleção inválido: {col}")

In [7]:
def create_country_document(cursor):
	query = 'SELECT * FROM PAIS;'
	cursor.execute(query)
	rows = cursor.fetchall()
	paises = []
	for row in rows:
		paises.append({
						'sigla': row[0],
						'nome': row[1],
						'continente': row[2],
						'estatisticas': []
					})
	# Grupos
	for pais in paises:
		# if pais['sigla'] != 'AFG':
		# 	continue
		query = f'''SELECT SIGLA
		FROM
		GRUPO G INNER JOIN PAIS_GRUPO PG
			ON G.ID_GRUPO = PG.GRUPO_ID_GRUPO
		WHERE PAIS_CODIGO = '{pais['sigla']}';'''
		cursor.execute(query)
		rows = cursor.fetchall()
		grupos = [row[0] for row in rows]
		if len(grupos) > 0:
			pais['grupos'] = grupos

		query = f'''SELECT ANO,
		VALOR_USD
		FROM PIB
		WHERE CODIGO_PAIS = '{pais['sigla']}'
		ORDER BY ANO;'''
		cursor.execute(query)
		rows = cursor.fetchall()

		for row in rows:
			ano, pib = row
			# print(ano, pib, pais['sigla'])
			geografias = []
			# Áreas e populações urbana, rural e total
			subquery = f'''SELECT TIPO,
			NUM_HABITANTES,
			AREA_KM2
			FROM
			AREA A INNER JOIN DEMOGRAFIA D
				ON D.AREA_ID = A.ID
			INNER JOIN DISTRIBUICAO_TERRITORIAL DT
				ON DT.AREA_ID = A.ID AND DT.PAIS_CODIGO = D.PAIS_CODIGO AND DT.ANO = D.ANO
			WHERE DT.PAIS_CODIGO = '{pais['sigla']}'
				AND DT.ANO = {ano}
			ORDER BY DT.ANO, TIPO;'''
			cursor.execute(subquery)
			subrows = cursor.fetchall()
			for sr in subrows:
				geografia = {
					'area_tipo': sr[0],
					'num_habitantes': sr[1],
					'area_km2': sr[2]
				}
				geografias.append(geografia)
			
			# Área florestal
			subquery = f'''SELECT TIPO,
			AREA_KM2
			FROM
			AREA A INNER JOIN DISTRIBUICAO_TERRITORIAL DT
				ON DT.AREA_ID = A.ID
			WHERE DT.PAIS_CODIGO = '{pais['sigla']}'
				AND DT.ANO = {ano}
			AND TIPO = 'Forest'
			ORDER BY DT.ANO, TIPO;'''
			cursor.execute(subquery)
			subrow = cursor.fetchall()
			for sr in subrow:
				geografia = {
					'area_tipo': sr[0],
					'area_km2': sr[1]
				}
				geografias.append(geografia)

			est = {
				'ano': row[0],
				'pib': row[1]
			}
			if geografias:
				est['geografias'] = geografias
			pais['estatisticas'].append(est)
	return paises


In [8]:
def create_gas_document(cursor):
	emissoes = []
	query = 'SELECT * FROM GAS;'
	cursor.execute(query)
	rows = cursor.fetchall()

	for row in rows:
		id, nome, formula = row
		emissao = {
			'nome': nome,
			'formula': formula,
			'medicoes': []
		}

		subquery = f'''SELECT PAIS_CODIGO,
		ANO,
		EMISSAO_MTCO2
		FROM EMISSAO_GAS
		WHERE GAS_ID_GAS = {id};'''
		cursor.execute(subquery)
		subrows = cursor.fetchall()

		for sr in subrows:
			cod_pais, ano, valor = sr
			emissao['medicoes'].append({
				'cod_pais': cod_pais,
				'ano': ano,
				'valor_mtco2': valor
			})
		
		emissoes.append(emissao)

	return emissoes

In [9]:
def create_energy_document(cursor):
	energias = []

	query = 'SELECT * FROM INTERACAO_ENERGETICA;'
	cursor.execute(query)
	rows = cursor.fetchall()

	for row in rows:
		id_interacao, interacao, unidade = row
		energia = {
			'interacao': interacao,
			'unidade': unidade,
			'fontes': []
		}
		
		query = 'SELECT * FROM ENERGIA;'
		cursor.execute(query)
		subrows = cursor.fetchall()

		for sr in subrows:
			id_energia, nome = sr
			fonte = {
				'nome': nome,
				'medicoes': []
			}
			query = f'''SELECT CODIGO_PAIS,
			ANO,
			VALOR
			FROM RELATORIO_ENERGETICO
			WHERE ID_INTERACAO = {id_interacao}
				AND ID_ENERGIA = {id_energia};'''
			cursor.execute(query)
			registros = cursor.fetchall()

			for reg in registros:
				cod_pais, ano, valor = reg
				fonte['medicoes'].append({
					'cod_pais': cod_pais,
					'ano': ano,
					'valor': valor
				})

			energia['fontes'].append(fonte)

		energias.append(energia)

	return energias

In [10]:
def populate_mongo(db):
	pg_conn = create_pg_connection()
	cursor = pg_conn.cursor()

	entries = {'pais': create_country_document(cursor),
		'emissaoGas': create_gas_document(cursor),
		'energia': create_energy_document(cursor)}
	
	end_pg_connection(pg_conn)
	
	for col in entries.keys():

		if col not in db.list_collection_names():
			db.create_collection(col)

		create_tables(db, col)
		db[col].delete_many({})
		db[col].insert_many(entries[col])

		print(f'> Coleção {col} <')
		# Exibindo os documentos inseridos
		for doc in db[col].find({}, {'_id': 0}):
			pprint(doc)

In [11]:
populate_tables = False
client = create_mongo_connection()
db = client["projeto_mc536"]

if populate_tables:
	populate_mongo(db)

end_mongo_connection(client)

Exception: ('The following error occurred: ', ConfigurationError('All nameservers failed to answer the query _mongodb._tcp.dgcluster.se4dgsc.mongodb.net. IN SRV: Server Do53:127.0.0.53@53 answered SERVFAIL'))

In [ ]:
def execute_pipeline(collection, pipeline, num=0):
    """Executa um pipeline de agregação MongoDB e retorna os resultados."""
    try:
        resultados = list(collection.aggregate(pipeline))
        if not resultados:
            print("Nenhum resultado encontrado.")
            return None

        df = pd.DataFrame(resultados)

        # Reorganiza as colunas com nome_pais primeiro, se existir
        col_order = ["nome_pais", "ano", "grupo", "energia_total", "unidade", "cod_pais"]
        col_order = [col for col in col_order if col in df.columns]
        df = df[col_order + [col for col in df.columns if col not in col_order]]

        pd.set_option('display.float_format', '{:.2f}'.format)

        print(df.to_string(index=False))

        if num != 0:
            df.to_csv(f"resultados_mongodb/resultado_pipeline_{num}.csv", index=False)

        print("Pipeline executado com sucesso.")
        return df
    except Exception as e:
        print(f"Erro ao executar pipeline: {e}")
        return None

# Query 1: Porcentagem de energia renovável produzida por grupo econômico e ano
Detecta a porcentagem renovável da energia total gerada por grupo econômico a cada ano.

In [12]:
pipeline = [
	{"$match": {"interacao": "Electricity generation"}},
    {"$unwind": "$fontes"},
    {"$unwind": "$fontes.medicoes"},
    {"$group": {
        "_id": {
            "cod_pais": "$fontes.medicoes.cod_pais",
            "ano": "$fontes.medicoes.ano"
        },
        "energia_renovavel": {
            "$sum": {
                "$cond": [
                    {"$in": ["$fontes.nome", ["Coal", "Gas", "Nuclear", "Other Fossil"]]},
                    0,
                    "$fontes.medicoes.valor"
                ]
            }
        },
        "energia_total": {"$sum": "$fontes.medicoes.valor"}
    }},
    {"$lookup": {
        "from": "pais",
        "localField": "_id.cod_pais",
        "foreignField": "sigla",
        "as": "pais_info"
    }},
    { "$unwind": "$pais_info" },
    { "$unwind": "$pais_info.grupos" },
    { "$group": {
        "_id": {
            "grupo": "$pais_info.grupos",
            "ano": "$_id.ano"
        },
        "energia_total": { "$sum": "$energia_total" },
        "energia_renovavel": { "$sum": "$energia_renovavel" }
    }},
    { "$addFields": {
        "porcentagem_renovavel": {
            "$cond": [
                { "$gt": ["$energia_total", 0] },
                {
                    "$multiply": [
                        { "$divide": ["$energia_renovavel", "$energia_total"] },
                        100
                    ]
                },
                None
            ]
        }
    }},
    { "$project": {
        "_id": 0,
        "grupo": "$_id.grupo",
        "ano": "$_id.ano",
        "porcentagem_renovavel": 1
    }},
    { "$sort": {
        "ano": 1,
        "grupo": 1
    }}
]
client = create_mongo_connection()
db = client["projeto_mc536"]
execute_pipeline(db['energia'], pipeline, 1)
end_mongo_connection(client)

Exception: ('The following error occurred: ', ConfigurationError('All nameservers failed to answer the query _mongodb._tcp.dgcluster.se4dgsc.mongodb.net. IN SRV: Server Do53:127.0.0.53@53 answered SERVFAIL'))

# Query 2: PIB por energia (per capita)
Compara os valores de Produto Interno Bruto per capita e energia per capita por país.

In [ ]:
pipeline = [
    { "$match": { "interacao": {"$in": ["Electricity generation", "Electricity demand"]} } },
    { "$unwind": "$fontes" },
    { "$unwind": "$fontes.medicoes" },
    {
        "$group": {
            "_id": {
                "cod_pais": "$fontes.medicoes.cod_pais",
                "ano": "$fontes.medicoes.ano",
            },
            "total_producao": { 
                "$sum": { 
                    "$cond": [{"$eq": ["$interacao", "Electricity generation"]},
                              "$fontes.medicoes.valor",
                              0]
                }
            },
            "total_demanda": {
                "$sum": {
                    "$cond": [{"$eq": ["$interacao", "Electricity demand"]},
                              "$fontes.medicoes.valor",
                              0]
                }
            }
        }
    },
    { "$match": {"_id.ano": {"$lt": 2023}}},
    {"$lookup": {
            "from": "pais",
            "localField": "_id.cod_pais",
            "foreignField": "sigla",
            "as": "pais_info"
    	}
    },
    { "$unwind": "$pais_info" },
    { "$addFields": {
            "estatistica_ano": {
                "$filter": {
                    "input": "$pais_info.estatisticas",
                    "as": "est",
                    "cond": { "$eq": ["$$est.ano", "$_id.ano"] }
                }
            }
        }
    },
    { "$unwind": "$estatistica_ano" },
    { "$addFields": {
        "populacao_total": {
            "$sum": {
                "$map": {
                    "input": "$estatistica_ano.geografias",
					"as": "geo",
					"in": "$$geo.num_habitantes"
				}
			}
		}
	}},

    # 7. Calcular produção per capita
    { "$addFields": {
		"producao_per_capita": {
			"$cond": [
				{ "$gt": ["$populacao_total", 0] },
				{"$multiply": [
					{ "$divide": ["$total_producao", "$populacao_total"] },
					1e9
				]},
				None
			]
		},
		"demanda_per_capita": {
			"$cond": [
				{"$gt": ["$populacao_total", 0]},
				{"$multiply": [
					{"$divide": ["$total_demanda", "$populacao_total"]},
					1e9
				]},
				None
			]                          
		},
		"pib_per_capita": {
			"$cond": [
				{ "$gt": ["$populacao_total", 0] },
				{ "$divide": ["$estatistica_ano.pib", "$populacao_total"] },
				None
			]
		}
	}},
    # 8. Projeção final
    { "$project": {
		"_id": 0,
		"ano": "$_id.ano",
        "nome_pais": "$pais_info.nome",
        "producao_per_capita": 1,
		"demanda_per_capita": 1,
		"pib_per_capita": 1
	}},
    { "$sort": {
		"nome_pais": 1,
		"ano": 1
	}}
    ]
client = create_mongo_connection()
db = client["projeto_mc536"]
execute_pipeline(db['energia'], pipeline, 2)
end_mongo_connection(client)

# Query 3: Relação entre matriz energética e emissão de co2 per capita, por país e ano
Compara os valores, buscando encontrar relação entre geração de energias não renováveis e emissão de co2. O critério de apresentação dos dados é por ordem decrescente de emissão per capita de co2.

In [13]:
ano_escolhido = 2010
pipeline = [
    # Selecionar somente dados de produção de energia do ano escolhido
	{"$match": {
        "interacao": "Electricity generation",
    }},
    {"$unwind": "$fontes"},
    {"$unwind": "$fontes.medicoes"},
    {"$match": {
        "fontes.medicoes.ano": ano_escolhido,
        "fontes.medicoes.valor": {"$gt": 0}
    }},
    # Calcular o total de energia produzida, mantendo as produções individuais
    {"$group": {
        "_id": {
            "cod_pais": "$fontes.medicoes.cod_pais",
            "ano": "$fontes.medicoes.ano",
            "tipo_energia": "$fontes.nome"
        },
        "valor": { "$sum": "$fontes.medicoes.valor" }
    }},
    {"$group": {
        "_id": {
            "cod_pais": "$_id.cod_pais",
            "ano": "$_id.ano"
        },
        "total": { "$sum": "$valor" },
        "detalhes": {
            "$push": {
                "tipo_energia": "$_id.tipo_energia",
                "valor": "$valor"
            }
        }
    }},
    # Join com o documento emissaoGas (Faz uma query para somente obter os dados de CO2)
    {"$lookup": {
        "from": "emissaoGas",
        "let": {
            "cod_pais": "$_id.cod_pais",
            "ano": "$_id.ano"
        },
        "pipeline": [
            {"$match": { "$expr": { "$eq": ["$nome", "Carbon dioxide"] } } },
            {"$unwind": "$medicoes" },
            {"$match": {
                "$expr": {
                    "$and": [
                    {"$eq": ["$medicoes.cod_pais", "$$cod_pais"] },
                    {"$eq": ["$medicoes.ano", "$$ano"] }
                    ]
                }
            }}
        ],
        "as": "emissao_info"
    }},
    {"$unwind": "$emissao_info"},
    # Join com o documento de países
    {"$lookup": {
        "from": "pais",
        "localField": "_id.cod_pais",
        "foreignField": "sigla",
        "as": "pais_info"
    }},
    # Obter o número de habitantes do país
    {"$unwind": "$pais_info" },
    {"$unwind": "$pais_info.estatisticas"},
    {"$match": {"pais_info.estatisticas.ano": ano_escolhido}},
    {"$unwind": "$pais_info.estatisticas.geografias"},
    {"$match": {"pais_info.estatisticas.geografias.area_tipo": "Surface"}},
    
    #Separar os tipos de energia
    {"$unwind": "$detalhes"},
    
    {"$project": {
        "_id": 0,
        "pais": "$pais_info.nome",
        "ano": "$_id.ano",
        "fonte": "$detalhes.tipo_energia",
        "producao_energia_porcentagem": {
            "$round": [
                {"$multiply": [{"$divide": ["$detalhes.valor", "$total"]},100]
                    },2]
        },
        "emissao_co2_per_capita": {
            "$round": [
                {"$cond": [
                    {"$gt": ["$pais_info.estatisticas.geografias.num_habitantes", 0]},
                    {"$multiply": [
                        {"$divide": ["$emissao_info.medicoes.valor_mtco2", "$pais_info.estatisticas.geografias.num_habitantes"]},
                        100000000
                        ]
                    },
                    None
                    ]
                }
            ,4]
        }
    }},
    # Ordenar
    {"$sort": {
        "emissao_co2_per_capita": -1,
        "pais": 1,
        "producao_energia_porcentagem": -1
    }}
]
client = create_mongo_connection()
db = client["projeto_mc536"]
execute_pipeline(db['energia'], pipeline, 3)
end_mongo_connection(client)

Exception: ('The following error occurred: ', ConfigurationError('All nameservers failed to answer the query _mongodb._tcp.dgcluster.se4dgsc.mongodb.net. IN SRV: Server Do53:127.0.0.53@53 answered SERVFAIL'))

# Query 4: Energia gerada por km² de área florestal
Traça relação entre geração de energia e área florestal no ano de 2015.

In [ ]:
pipeline = [
    {"$match": {"interacao": "Electricity generation"}},
    {"$unwind": "$fontes"},
    {"$unwind": "$fontes.medicoes"},
    {"$match": {"fontes.medicoes.ano": 2015}},
    {"$group": {
        "_id": "$fontes.medicoes.cod_pais",
        "energia_total": {"$sum": "$fontes.medicoes.valor"},
        "unidade": {"$first": "$unidade"},
        "ano": {"$first": "$fontes.medicoes.ano"}
    }},
    {"$lookup": {
        "from": "pais",
        "localField": "_id",
        "foreignField": "sigla",
        "as": "pais_info"
    }},
    {"$unwind": "$pais_info"},
    {"$unwind": "$pais_info.estatisticas"},
    {"$match": {"$expr": {"$eq": ["$pais_info.estatisticas.ano", "$ano"]}}},
    {"$unwind": "$pais_info.estatisticas.geografias"},
    {"$match": {"pais_info.estatisticas.geografias.area_tipo": "Forest"}},
    {"$match": {"pais_info.estatisticas.geografias.area_km2": {"$gt": 0}}},
    {"$project": {
        "_id": 0,
        "pais": "$pais_info.nome",
        "ano": "$ano",
        "mwh_por_km2_florestal": {
            "$multiply": [
                {
                "$divide": [
                    "$energia_total",
                    "$pais_info.estatisticas.geografias.area_km2"
                ]
                },
                1000000
                ]
        }
        
    }},
    {"$sort": {"pais": 1}}
]
client = create_mongo_connection()
db = client["projeto_mc536"]
execute_pipeline(db['energia'], pipeline, 4)
end_mongo_connection(client)

# Query 5: Porcentagem de energia renovável produzida vs porcentagem de população rural
Compara as taxas ordenando por áno e país.

In [ ]:
pipeline = [
    {"$match": {"interacao": "Electricity generation"}},
    {"$unwind": "$fontes"},
    {"$unwind": "$fontes.medicoes"},
    {"$group": {
        "_id": {
            "cod_pais": "$fontes.medicoes.cod_pais",
            "ano": "$fontes.medicoes.ano"
        },
        "energia_renovavel": {
            "$sum": {
                "$cond": [
                    {"$in": ["$fontes.nome", ["Coal", "Gas", "Nuclear", "Other Fossil"]]},
                    0,
                    "$fontes.medicoes.valor"
                ]
            }
        },
        "energia_total": {"$sum": "$fontes.medicoes.valor"}
    }},
    {"$lookup": {
        "from": "pais",
        "localField": "_id.cod_pais",
        "foreignField": "sigla",
        "as": "pais_info"
    }},
    {"$unwind": "$pais_info"},
    {"$unwind": "$pais_info.estatisticas"},
    {"$unwind": "$pais_info.estatisticas.geografias"},
    {"$group": {
        "_id": {
            "cod_pais": "$_id.cod_pais",
            "ano": "$_id.ano"
        },
        "energia_renovavel": {"$first": "$energia_renovavel"},
        "energia_total": {"$first": "$energia_total"},
        "pop_rural": {
            "$sum": {
                "$cond": [
                    {"$eq": ["$pais_info.estatisticas.geografias.area_tipo", "Rural"]},
                    "$pais_info.estatisticas.geografias.num_habitantes",
                    0
                ]
            }
        },
        "pop_total": {
            "$sum": {
                "$cond": [
                    {"$in": ["$pais_info.estatisticas.geografias.area_tipo", ["Rural", "Urban"]]},
                    "$pais_info.estatisticas.geografias.num_habitantes",
                    0
                ]
            }
        },
        "pais": {"$first": "$pais_info.nome"}
    }},
    {"$project": {
        "_id": 0,
        "ano": "$_id.ano",
        "pais": "$pais",
        "%_energia_renovavel": {
            "$multiply": [
                {"$divide": ["$energia_renovavel", "$energia_total"]},
                100
            ]
        },
        "%_pop_rural": {
            "$multiply": [
                {"$divide": ["$pop_rural", "$pop_total"]},
                100
            ]
        }
    }},
    {"$sort": {"ano": 1, "pais": 1}}
]

client = create_mongo_connection()
db = client["projeto_mc536"]
execute_pipeline(db['energia'], pipeline, 5)
end_mongo_connection(client)